# Exploration Notebook
Scratchpad to explore the Free NBA API and DuckDB schema.

In [ ]:
# heartbeat
print("Kernel is alive")

# Sanity check API call

In [ ]:
import httpx, os, json

API_KEY = os.getenv("BALLDONTLIE_API_KEY")
BASE = "https://api.balldontlie.io/v1"
HEADERS = {"Authorization": API_KEY}

date_str = "2025-06-22"

with httpx.Client(timeout=httpx.Timeout(10.0, connect=5.0)) as client:
    r = client.get(f"{BASE}/games", headers=HEADERS, params={"dates[]": date_str, "per_page": 100, "page": 1})
    print("HTTP", r.status_code)
    # If it’s JSON, show a quick summary; otherwise show text
    try:
        payload = r.json()
        print("keys:", list(payload.keys()))
        print("count in this page:", len(payload.get("data", [])))
        print("meta:", payload.get("meta"))
    except Exception:
        print(r.text[:200])


# 1. Get data back from the API for games on a given date e.g. 2025-04-27


In [ ]:
import pandas as pd, time

def fetch_games_on(date_str: str, per_page: int = 100) -> list[dict]:
    all_rows, page = [], 1
    with httpx.Client(timeout=httpx.Timeout(10.0, connect=5.0)) as client:
        while True:
            r = client.get(f"{BASE}/games", headers=HEADERS,
                           params={"dates[]": date_str, "per_page": per_page, "page": page})
            r.raise_for_status()
            payload = r.json()
            data = payload.get("data", [])
            meta = payload.get("meta", {}) or {}
            all_rows.extend(data)
            if page >= (meta.get("total_pages") or 1) or not data:
                break
            page += 1
    return all_rows

def simplify(g):
    return {
        "game_id": g["id"],
        "date": g["date"][:10] if g.get("date") else None,
        "status": g.get("status"),
        "home_team": g["home_team"]["full_name"],
        "away_team": g["visitor_team"]["full_name"],
        "home_score": g.get("home_team_score"),
        "away_score": g.get("visitor_team_score"),
        "season": g.get("season"),
        "postseason": g.get("postseason"),
    }

target_date = "2025-04-27"
rows = fetch_games_on(target_date)
print(f"Found {len(rows)} games on {target_date}")

df = pd.DataFrame([simplify(g) for g in rows])
df.sort_values(["date", "game_id"], inplace=True, ignore_index=True)
df.head(10)


# 2. Returns data just for a single game specified by gameID

In [ ]:
import httpx, pandas as pd

# Re-use your API key and BASE/HEADERS defined earlier
API_KEY = "<your_api_key>"
BASE = "https://api.balldontlie.io/v1"
HEADERS = {"Authorization": API_KEY}

def fetch_game(game_id: int) -> dict:
    """Fetch details for a single game by ID."""
    with httpx.Client(timeout=httpx.Timeout(10.0, connect=5.0)) as client:
        r = client.get(f"{BASE}/games/{game_id}", headers=HEADERS)
        r.raise_for_status()
        return r.json()

def simplify_game(resp: dict) -> dict:
    """Extract key fields from the raw JSON (unwrap 'data' if present)."""
    # unravel the "data" block aka flatten it
    g = resp.get("data", resp)  # handle both raw object and wrapped
    return {
        "game_id": g["id"],
        "date": g.get("date", "")[:10],
        "status": g.get("status"),
        "season": g.get("season"),
        "postseason": g.get("postseason"),
        "home_team": g["home_team"]["full_name"],
        "away_team": g["visitor_team"]["full_name"],
        "home_score": g.get("home_team_score"),
        "away_score": g.get("visitor_team_score"),
    }


# Example: Orlando Magic vs Boston Celtics on 2025-04-27
example_id = 18422304
game = fetch_game(example_id)
print("Raw keys:", list(game.keys()))

row = simplify_game(game)
df = pd.DataFrame([row])
df


# Player totals (points) for a single game — BallDontLie /stats?game_ids[]=<id>
Gets a 401 as this requires a paid subscription

In [ ]:
import os, time
import httpx
import pandas as pd

# Reuse existing globals if present; otherwise define them here
try:
    API_KEY
    BASE
    HEADERS
except NameError:
    API_KEY = "<your_api_key>"  # or os.getenv("BALLDONTLIE_API_KEY")
    BASE = "https://api.balldontlie.io/v1"
    HEADERS = {"Authorization": API_KEY}    

def fetch_player_stats_for_game(game_id: int, per_page: int = 100) -> list[dict]:
    """Fetch all player stat rows for a game, following pagination."""
    all_rows, page = [], 1
    with httpx.Client(timeout=httpx.Timeout(10.0, connect=5.0)) as client:
        while True:
            r = client.get(
                f"{BASE}/stats",
                headers=HEADERS,
                params={"game_ids[]": game_id, "per_page": per_page, "page": page},
            )
            r.raise_for_status()
            payload = r.json()
            data = payload.get("data", payload)  # handle wrapped or bare
            meta = payload.get("meta", {}) or {}
            if isinstance(data, dict):
                # some rare cases return a single object
                data = [data]
            all_rows.extend(data)
            if page >= (meta.get("total_pages") or 1) or not data:
                break
            page += 1
    return all_rows

def player_points_dataframe(game_id: int) -> pd.DataFrame:
    """Aggregate total points per player for the game (exclude zero-point players)."""
    rows = fetch_player_stats_for_game(game_id)
    # Build tidy rows
    tidy = []
    for r in rows:
        player = r.get("player") or {}
        team = r.get("team") or {}
        tidy.append({
            "player_id": player.get("id"),
            "player_name": f"{player.get('first_name','')} {player.get('last_name','')}".strip(),
            "team_id": team.get("id"),
            "team_abbr": team.get("abbreviation"),
            "pts": r.get("pts") or 0,
        })
    df = pd.DataFrame(tidy)
    if df.empty:
        return df
    # Sum points per player, drop zero scorers, sort desc
    agg = (
        df.groupby(["player_id", "player_name", "team_id", "team_abbr"], dropna=False)["pts"]
          .sum()
          .reset_index(name="points_total")
    )
    agg = agg[agg["points_total"] > 0].sort_values(["points_total", "player_name"], ascending=[False, True]).reset_index(drop=True)
    return agg

# ==== Example usage ====
game_id = 18422304  # e.g., 2025-04-27 ORL vs BOS
player_points_df = player_points_dataframe(game_id)
print(f"Fetched {len(player_points_df)} scoring players for game_id={game_id}")
player_points_df.head(15)


# Get a mapping of the teams to use in the next steps

In [ ]:
# pip install requests python-dateutil
import os
import time
import requests
from typing import Dict, List, Tuple, Optional

API_BASE = "https://api.balldontlie.io/v1"
API_KEY = os.getenv("BALLDONTLIE_API_KEY", "<your_api_key>")
HEADERS = {"Authorization": API_KEY}

# Simple in-memory cache
_TEAM_CACHE = {
    "fetched_at": 0.0,
    "ttl_sec": 24 * 60 * 60,   # 24h
    "teams": [],               # raw team objects
    "index": {},               # str -> team_id (for quick lookup)
    "reverse": {}              # team_id -> canonical name
}

def _normalise(s: str) -> str:
    return " ".join(s.strip().lower().split())

def _build_index(teams: List[dict]) -> Tuple[Dict[str, int], Dict[int, str]]:
    """
    Build a flexible lookup index:
      - full_name (e.g., "Boston Celtics")
      - name (e.g., "Celtics")
      - abbreviation (e.g., "BOS")
      - city + name (e.g., "Boston Celtics")
    """
    idx: Dict[str, int] = {}
    rev: Dict[int, str] = {}

    for t in teams:
        tid = t["id"]
        abbr = t.get("abbreviation", "") or ""
        city = t.get("city", "") or ""
        name = t.get("name", "") or ""
        full = t.get("full_name", "") or f"{city} {name}".strip()

        rev[tid] = full

        candidates = {
            _normalise(full),
            _normalise(name),
            _normalise(abbr),
            _normalise(f"{city} {name}"),
            _normalise(city),  # sometimes people type just the city
        }

        for key in candidates:
            if not key:
                continue
            # If collision (e.g., "Los Angeles") occurs, keep it ambiguous for now.
            # We'll handle ambiguity at query time by collecting candidates.
            if key in idx and idx[key] != tid:
                # mark as ambiguous sentinel by storing -1
                idx[key] = -1
            else:
                idx[key] = tid

    return idx, rev

def _fetch_teams() -> List[dict]:
    resp = requests.get(f"{API_BASE}/teams", headers=HEADERS, timeout=30)
    resp.raise_for_status()
    payload = resp.json()
    return payload.get("data", [])

def _ensure_team_cache(force: bool = False) -> None:
    now = time.time()
    if force or (now - _TEAM_CACHE["fetched_at"] > _TEAM_CACHE["ttl_sec"]) or not _TEAM_CACHE["teams"]:
        teams = _fetch_teams()
        idx, rev = _build_index(teams)
        _TEAM_CACHE.update({
            "fetched_at": now,
            "teams": teams,
            "index": idx,
            "reverse": rev
        })

def list_teams() -> List[Tuple[int, str, str]]:
    """Convenience helper for a human-readable menu."""
    _ensure_team_cache()
    out = []
    for t in _TEAM_CACHE["teams"]:
        out.append((t["id"], t.get("full_name", f"{t.get('city','')} {t.get('name','')}".strip()), t.get("abbreviation","")))
    # sort nicely
    return sorted(out, key=lambda x: x[1])

def resolve_team(query: str) -> int:
    """
    Resolve a team by a flexible string:
      - "Boston Celtics" (full_name)
      - "Celtics" (name)
      - "BOS" (abbreviation)
      - "Boston", "Los Angeles", etc. (may be ambiguous)
    Handles ambiguity by attempting smart narrowing; otherwise raises ValueError with suggestions.
    """
    _ensure_team_cache()
    q = _normalise(query)
    idx = _TEAM_CACHE["index"]

    # Direct hit?
    if q in idx and idx[q] > 0:
        return idx[q]

    # If stored as ambiguous sentinel (-1) or not found, try substring search over full_name and city+name
    candidates = []
    for t in _TEAM_CACHE["teams"]:
        full = _normalise(t.get("full_name", ""))
        city_name = _normalise(f"{t.get('city','')} {t.get('name','')}")
        abbr = _normalise(t.get("abbreviation",""))
        name_only = _normalise(t.get("name",""))

        if q == full or q == city_name or q == abbr or q == name_only:
            candidates.append(t)
        elif q in full or q in city_name:
            candidates.append(t)

    # Unique?
    unique_ids = list({t["id"] for t in candidates})
    if len(unique_ids) == 1:
        return unique_ids[0]

    # No/Many → raise with helpful suggestions
    if not candidates:
        # Show a few closest by simple contains on tokens
        hints = ", ".join([t["full_name"] for t in _TEAM_CACHE["teams"][:5]])
        raise ValueError(f"No team match for '{query}'. Example teams: {hints}")
    else:
        opts = ", ".join(sorted({t["full_name"] for t in candidates}))
        raise ValueError(f"Ambiguous team '{query}'. Did you mean one of: {opts}")

def team_name(team_id: int) -> Optional[str]:
    _ensure_team_cache()
    return _TEAM_CACHE["reverse"].get(team_id)


# Check cache

In [ ]:
# Force refresh the cache (optional)
_ensure_team_cache(force=True)

# Peek at the whole cache dict
# Raw team objects (first 2 only for display)
# _TEAM_CACHE["teams"][:2]

# Lookup dictionary (sample keys)
# list(_TEAM_CACHE["index"].items())[:10]

# Reverse lookup (id → name)
_TEAM_CACHE["reverse"]



In [ ]:
# Force refresh the cache (optional)
_ensure_team_cache(force=True)

# Peek at the whole cache dict
_TEAM_CACHE.keys()


# Points 3 and 4 on the brief
## 3. Create an endpoint that allows one to download a CSV file with aggregated statistics by team
for a requested date period, start_date, end_date, e.g., 2022-01-31
## 4. The CSV file should contain: Team name, Median score, Percentage of games won. Additional
fields are allowed. (Stretch goal: put in home and away summaries)

In [ ]:
# pip install requests pandas python-dateutil numpy
import os
import math
import time
import requests
import pandas as pd
import numpy as np
from dateutil.parser import isoparse

API_BASE = "https://api.balldontlie.io/v1"
API_KEY = os.getenv("BALLDONTLIE_API_KEY", "<your_api_key>")  # <-- replace if not using env var
HEADERS = {"Authorization": API_KEY}

def fetch_games_for_team(team_id: int, start_date: str, end_date: str, per_page: int = 100, max_pages: int = 100):
    """
    Fetch all games for a given team and date range (inclusive). Handles pagination.
    Dates must be 'YYYY-MM-DD'.
    """
    # Basic validation to avoid accidental reversed dates
    sd = isoparse(start_date).date()
    ed = isoparse(end_date).date()
    if ed < sd:
        raise ValueError("end_date must be >= start_date")
        
    games = []
    page = 1
    while True:
        params = {
            "team_ids[]": team_id,
            "start_date": start_date,
            "end_date": end_date,
            "per_page": per_page,
            "page": page,
        }
        resp = requests.get(f"{API_BASE}/games", headers=HEADERS, params=params, timeout=30)
        resp.raise_for_status()
        payload = resp.json()
        data = payload.get("data", [])
        meta = payload.get("meta", {})
        games.extend(data)
        
        total_pages = meta.get("total_pages", 1)
        if page >= total_pages or page >= max_pages:
            break
        page += 1
        # be polite if rate-limited later
        time.sleep(0.05)
    return games

def compute_team_aggregates(games: list, team_id: int):
    if not games:
        return {
            "team_id": team_id,
            "team_name": None,
            "games_count": 0,
            "median_score": None,
            "win_pct": None,
            "home_games": 0,
            "home_win_pct": None,
            "away_games": 0,
            "away_win_pct": None,
        }, pd.DataFrame()

    rows = []
    team_name = None

    for g in games:
        home = g["home_team"]
        away = g["visitor_team"]
        hs = g["home_team_score"]
        vs = g["visitor_team_score"]
        date = g["date"]

        if home["id"] == team_id:
            team_side = "home"
            team_points, opp_points = hs, vs
            opp_team = away.get("full_name", away.get("name"))
            if team_name is None:
                team_name = home.get("full_name", home.get("name"))
        elif away["id"] == team_id:
            team_side = "away"
            team_points, opp_points = vs, hs
            opp_team = home.get("full_name", home.get("name"))
            if team_name is None:
                team_name = away.get("full_name", away.get("name"))
        else:
            continue

        rows.append({
            "date": date,
            "team_id": team_id,
            "team_name": team_name,
            "side": team_side,
            "team_points": team_points,
            "opp_points": opp_points,
            "won": team_points > opp_points,
            "opponent": opp_team,
        })

    df = pd.DataFrame(rows)
    if df.empty:
        return {
            "team_id": team_id,
            "team_name": team_name,
            "games_count": 0,
            "median_score": None,
            "win_pct": None,
            "home_games": 0,
            "home_win_pct": None,
            "away_games": 0,
            "away_win_pct": None,
        }, df
    
    games_count = len(df)
    median_score = float(np.median(df["team_points"])) if games_count > 0 else None
    win_pct = round(100.0 * df["won"].mean(), 2) if games_count > 0 else None
    
    # Stretch: home/away splits
    home_df = df[df["side"] == "home"]
    away_df = df[df["side"] == "away"]
    home_games = len(home_df)
    away_games = len(away_df)
    home_win_pct = round(100.0 * home_df["won"].mean(), 2) if home_games > 0 else None
    away_win_pct = round(100.0 * away_df["won"].mean(), 2) if away_games > 0 else None
    
    summary = {
        "team_id": team_id,
        "team_name": team_name,
        "games_count": games_count,
        "median_score": round(median_score, 2) if median_score is not None else None,
        "win_pct": win_pct,
        "home_games": home_games,
        "home_win_pct": home_win_pct,
        "away_games": away_games,
        "away_win_pct": away_win_pct,
    }
    return summary, df

def aggregate_to_csv(team_id: int, start_date: str, end_date: str, out_path: str = None):
    """
    High-level convenience:
    - fetch games
    - compute summary
    - write a 1-row CSV of aggregated stats
    - also return (summary_dict, per_game_df)
    """
    games = fetch_games_for_team(team_id, start_date, end_date)
    summary, df_games = compute_team_aggregates(games, team_id)
    # Default path
    if out_path is None:
        out_path = f"team_{team_id}_{start_date}_to_{end_date}_summary.csv"
    # Single-row CSV for the spec
    pd.DataFrame([summary]).to_csv(out_path, index=False)
    return summary, df_games, out_path

# Example: user supplies a name or abbreviation (human-friendly)
#tid = resolve_team("BOS")              # -> 2 (Boston Celtics)
#tid = resolve_team("Boston Celtics") # also works
tid = resolve_team("bulls")        # also works

# Now call your existing aggregator
summary, per_game_df, csv_path = aggregate_to_csv(
    tid, start_date="2025-01-30", end_date="2025-04-01"
)
summary, csv_path

